# Mounting The GDrive 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
audio_path = '/content/drive/My Drive/Dataset/Actor_01/03-01-01-01-01-01-01.wav'

In [5]:

import IPython.display as ipd
import librosa
import librosa.display
import pandas as pd
import numpy as np


In [6]:
x,sr = librosa.load(audio_path)
ipd.Audio(audio_path)

# Importing The RAVDESS Dataset From The GDrive

In [7]:
dataset = '/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/'

In [8]:
import os

In [9]:
actor_folders = os.listdir(dataset)
actor_folders.sort() 
actor_folders

['Actor_01',
 'Actor_02',
 'Actor_03',
 'Actor_04',
 'Actor_05',
 'Actor_06',
 'Actor_07',
 'Actor_08',
 'Actor_09',
 'Actor_10',
 'Actor_11',
 'Actor_12',
 'Actor_13',
 'Actor_14',
 'Actor_15',
 'Actor_16',
 'Actor_17',
 'Actor_18',
 'Actor_19',
 'Actor_20',
 'Actor_21',
 'Actor_22',
 'Actor_23',
 'Actor_24']

# Creating A Dataframe For The Given Data


In [10]:
emotions = []
gender = []
actor_ID = []
file_path = []
for i in actor_folders:
    filename = os.listdir(dataset + i) #iterate over Actor folders
    for f in filename: # go through files in the corresponding Actor's folder
        part = f.split('.')[0].split('-')#We split the file name initially as['03-01-01-01-01-01-01','wav'];Now,the 0th element is converted as ['03','01','01','01','01','01','01']
        emotions.append(int(part[2])) #Since,the second element represents the emotion of the actor.
        actor_ID.append(int(part[6])) #Since,the sixth element represents the ID of the actor.
        bg = int(part[6])
        if bg%2 == 0:
            bg = "female" #Since,even IDs correspond to females.
        else:
            bg = "male"  #Since,odd IDs correspond to females.
        gender.append(bg)
        file_path.append(dataset + i + '/' + f)

In [11]:
#Creating a dataframe:
audio_df = pd.DataFrame(emotions)
audio_df = audio_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor_ID)],axis=1)
audio_df.columns = ['Gender','Emotion','Actor_ID']
audio_df = pd.concat([audio_df,pd.DataFrame(file_path, columns = ['Path'])],axis=1)
audio_df

,Gender,Emotion,Actor_ID,Path
0,male,angry,1,/content/drive/MyDrive/Dataset/audio_speech_ac...
1,male,calm,1,/content/drive/MyDrive/Dataset/audio_speech_ac...
2,male,sad,1,/content/drive/MyDrive/Dataset/audio_speech_ac...
3,male,fear,1,/content/drive/MyDrive/Dataset/audio_speech_ac...
4,male,sad,1,/content/drive/MyDrive/Dataset/audio_speech_ac...
...,...,...,...,...
1435,female,surprise,24,/content/drive/MyDrive/Dataset/audio_speech_ac...
1436,female,surprise,24,/content/drive/MyDrive/Dataset/audio_speech_ac...
1437,female,disgust,24,/content/drive/MyDrive/Dataset/audio_speech_ac...
1438,female,disgust,24,/content/drive/MyDrive/Dataset/audio_speech_ac...


In [12]:
pd.set_option('display.max_colwidth', -1)

audio_df.sample(10)

,Gender,Emotion,Actor_ID,Path
141,male,calm,3,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_03/03-01-02-01-01-01-03.wav
1067,female,disgust,18,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_18/03-01-07-02-01-02-18.wav
115,female,disgust,2,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_02/03-01-07-01-02-01-02.wav
1322,male,calm,23,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_23/03-01-02-02-01-02-23.wav
311,female,disgust,6,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_06/03-01-07-02-01-01-06.wav
348,female,disgust,6,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_06/03-01-07-02-02-01-06.wav
527,male,surprise,9,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_09/03-01-08-01-02-01-09.wav
400,male,disgust,7,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_07/03-01-07-01-01-01-07.wav
362,male,happy,7,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_07/03-01-03-02-01-02-07.wav
919,female,calm,16,/content/drive/MyDrive/Dataset/audio_speech_actors_01-24/Actor_16/03-01-02-01-01-02-16.wav


# Uploading The File As A CSV File

In [13]:
# EXPORT TO CSV(Saving the dataframe as a csv file to the gdrive under the Dataset folder.)
audio_df.to_csv('/content/drive/MyDrive/Dataset/CSV_dataset')

# Feature Extraction Using MEL

In [14]:
# ITERATE OVER ALL AUDIO FILES AND EXTRACT LOG MEL SPECTROGRAM MEAN VALUES INTO DF FOR MODELING 
df = pd.DataFrame(columns=['mel_spectrogram'])

counter=0

for index,path in enumerate(audio_df.Path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)

    #get the mel-scaled spectrogram and transform both the y-axis (frequency) to log scale, and the “color” axis (amplitude) to Decibels.
    spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000) 
    db_spec = librosa.power_to_db(spectrogram)

    #temporally average spectrogram
    log_spectrogram = np.mean(db_spec, axis = 0)
  

    df.loc[counter] = [log_spectrogram]
    counter=counter+1   
#259 new Features have been created using the MEL Spectogram.
print(len(df))
df.head()

1440


,mel_spectrogram
0,"[-64.17679, -64.39374, -64.444954, -64.26804, -64.303825, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.46815, -64.447296, -64.4122, -64.400055, -64.46363, -64.46701, -64.44746, -64.398, -64.213005, -64.223236, -64.3529, -64.37738, -63.900597, -63.455616, -63.70556, -63.493465, -63.146347, -63.393356, -64.024376, -64.01411, -64.01472, -62.16667, -53.427326, -53.112213, -56.749325, -59.260284, -63.197086, -39.888687, -29.309977, -23.734594, -22.996405, -22.082584, -21.821333, -22.878685, -24.086143, -25.910528, -27.233507, -28.2812, -28.47165, -27.822582, -25.86707, -23.51746, -22.555597, -21.906961, -22.342373, -24.088245, -25.46311, -27.872072, -32.053486, -36.408924, -39.520267, -41.121758, -43.039547, -39.19178, -33.291317, -27.97979, -26.682621, -28.951216, -31.153667, -32.23939, -33.277252, -33.3202, -32.63985, -31.742908, -33.313877, -36.443226, -37.307518, -40.296413, -46.544678, -47.72739, -43.08766, -38.47714, -35.703156, -33.48515, -32.37977, -31.097954, -30.754461, -28.716875, -27.440126, -26.975042, -29.101736, -31.4359, -32.79103, ...]"
1,"[-78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.43884, -78.4542, -78.4542, -78.4542, -78.41042, -78.45068, -78.4542, -78.42379, -78.20893, -78.26179, -78.393005, -78.181946, -76.54556, -77.80522, -78.35667, -78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.35812, -78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.4542, -78.30332, -77.40237, -76.72194, -74.78916, -73.5713, -71.05219, -71.32608, -74.50437, -51.601734, -40.75338, -37.786884, -35.510395, -33.946377, -33.06304, -32.153732, -30.557005, -30.412754, -31.12473, -31.028309, -31.162865, -31.399376, -31.185083, -32.001812, -32.653805, -33.62147, -33.717075, -33.821217, -34.81197, -35.28008, -36.12838, -37.141144, -38.139374, -39.70676, -43.652508, -47.02291, -49.263157, -49.088577, -44.685097, -41.291615, -40.862713, -42.37775, -43.935345, -42.301487, -39.064354, -37.1027, -36.956676, -37.50367, -38.67591, -39.754787, -39.825855, -40.14709, -40.670406, -40.860516, -41.18104, -42.499855, -42.641098, -43.528965, -43.827126, -42.342186, -38.96177, -40.174595, -41.470406, -38.25139, -34.112953, -32.843117, ...]"
2,"[-69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.70702, -69.22943, -68.50265, -68.04834, -66.714714, -66.56919, -66.5077, -66.30121, -66.074875, -67.55204, -68.678, -69.2261, -69.64466, -69.231735, -67.990654, -54.78028, -41.82301, -40.31695, -38.574577, -34.386005, -31.839472, -30.80399, -30.741735, -30.99812, -30.938948, -31.409452, -31.762432, -32.306732, -32.929276, -33.52696, -34.19554, -34.28472, -33.730526, -32.409443, -31.965248, -32.31671, -32.93777, -34.44699, -36.682354, -40.420925, -44.35431, -46.938892, -49.426384, -51.04467, -49.811943, -46.573715, -40.882896, -38.773422, -37.556614, -37.36039, -37.84805, -37.636673, -37.82189, -38.30929, -39.314316, -40.786964, -42.00275, -43.013313, -46.494576, -49.26306, -50.364277, -53.837505, -54.170353, -48.35747, -39.066757, -34.441166, -33.487663, -33.95173, -35.2858, -36.58439, -36.500725, -36.121727, -36.37019, ...]"
3,"[-73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.25525, -73.08632, -73.234695, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2803, -73.2766, -72.74323, -71.09078, -70.32488, -69.20192, -68.22371, -67.45397, -66.758484, -65.69483, -65.750854, -66.72477, -66.233826, -64.321724, -60.120052, -56.59815, -57.859257, -64.78188, -70.005554, -72.02886, -72.05144, -66.42027, -46.804337, -38.68785, -40.12812, -42.07857, -35.87358, -31.867702, -31.069366,

In [15]:
df.tail(5)

,mel_spectrogram
1435,"[-45.199844, -43.82312, -44.344078, -45.735817, -46.09549, -45.70201, -45.73389, -46.317333, -47.424995, -48.18975, -48.735367, -49.609917, -51.114033, -51.741745, -52.354424, -54.285316, -55.315136, -56.21937, -57.399914, -58.40338, -58.453133, -59.335815, -61.146862, -61.302704, -61.302704, -61.302704, -61.302704, -61.302704, -61.302704, -61.302704, -61.302704, -61.302704, -61.302704, -61.24344, -61.220222, -61.302704, -61.302704, -61.297314, -61.11591, -60.8, -60.77184, -60.81804, -35.838345, -22.939175, -18.907555, -18.615974, -18.044632, -19.236794, -20.700361, -22.053825, -22.831844, -22.125698, -20.603863, -18.693386, -17.710058, -18.467213, -19.365517, -19.294952, -17.838789, -16.966835, -18.129599, -20.528946, -23.195402, -26.6991, -30.6141, -36.81126, -39.28417, -39.47635, -39.062332, -33.148273, -28.453323, -28.692165, -31.438593, -32.630768, -32.336945, -30.512367, -29.220499, -29.22226, -27.886509, -25.618742, -25.172234, -27.441526, -28.867373, -29.312807, -29.23394, -29.609339, -28.110096, -26.81874, -26.665798, -27.538574, -30.08525, -32.81745, -33.48518, -34.063946, -35.153004, -35.127666, -35.037117, -34.617847, -34.426197, -34.83367, ...]"
1436,"[-68.403206, -68.403206, -68.403206, -68.403206, -68.403206, -68.403206, -68.403206, -68.403206, -68.39933, -67.85737, -67.09028, -66.30181, -66.843895, -64.562645, -62.644104, -61.08439, -51.850674, -46.687138, -46.233536, -47.738266, -50.351227, -51.099136, -51.55874, -51.873337, -51.094936, -52.01176, -53.007988, -53.324146, -54.20493, -56.11591, -58.938377, -60.911224, -62.974407, -65.346306, -66.77731, -66.88043, -67.60718, -67.390724, -67.21244, -67.43361, -66.86794, -66.04039, -41.059517, -28.608719, -25.292511, -26.05246, -26.391317, -26.524662, -27.728998, -28.218582, -28.477669, -28.208939, -27.19627, -25.770796, -23.969816, -23.81917, -24.459377, -25.094645, -26.49069, -29.775993, -34.39015, -34.832424, -34.017544, -32.158573, -32.605686, -33.79852, -33.571957, -34.697254, -34.184093, -31.201765, -31.05768, -32.83641, -32.340733, -30.596117, -30.220142, -31.881308, -33.734722, -36.262928, -36.762638, -37.759594, -37.63619, -37.352978, -37.850567, -36.969803, -35.912235, -34.61213, -31.636143, -29.126024, -30.319561, -32.30253, -30.747065, -30.610636, -34.082203, -33.746162, -29.717089, -29.15592, -31.856953, -30.911425, -32.58008, -35.849926, ...]"
1437,"[-68.237564, -67.37217, -66.39064, -66.67907, -67.21122, -66.47518, -65.98764, -64.50627, -63.091972, -62.79904, -62.180588, -62.196743, -62.990387, -63.585327, -63.532772, -64.269554, -55.129112, -53.41817, -60.28062, -62.33669, -64.34196, -66.67321, -68.57586, -69.06307, -69.53056, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.5409, -69.522964, -69.34986, -68.874985, -68.28364, -48.151608, -35.829742, -32.85075, -31.774574, -31.69519, -32.16299, -32.048813, -32.759552, -33.703457, -34.266243, -34.744892, -34.588085, -33.802013, -34.045555, -33.82419, -34.22323, -34.30419, -33.180824, -31.545694, -33.294876, -33.86307, -34.853283, -35.81161, -35.493793, -36.525414, -38.627686, -38.931286, -40.737717, -43.347107, -45.24238, -46.147522, -47.710297, -48.3077, -49.22619, -46.84356, -44.269077, -42.59426, -41.993397, -40.6192, -39.795975, -40.85453, -41.386864, -41.393814, -41.934322, -42.585598, -40.679256, -38.12509, -38.485588, -39.72423, -39.87061, -44.126175, -42.32663, -42.01108, -42.86875, -41.66372, -39.513733, -37.70224, ...]"
1438,"[-59.609074, -59.128216, -59.562916, -61.30292, -61.61981, -61.77404, -62.13151, -62.654728, -62.494194, -62.51393, -62.470318, -62.60258, -62.4045, -62.136993, -61.945717, -61.07031, -60.944786, -60.753075, -60.514732, -60.632572, -61.278114, -61.728016, -61.656292, -62.30469, -62.698635, -62.698635, -62.698635, -62.698635, -62.698635, -62.698635, -62.698635, -62.698635, -62.698635, -62.698635, -62.66831, -62.698635, -62.19058, -61.49057, -61.036694, -60.5898, -6

In [16]:
print(counter)
print(spectrogram.shape)
print(db_spec.shape)
print(log_spectrogram.shape)
len(log_spectrogram)
log_spectrogram

1440
(128, 259)
(128, 259)
(259,)


array([-59.71531 , -59.487293, -58.765045, -58.07815 , -57.8658  ,
       -57.900166, -56.573082, -57.192883, -57.220207, -56.61677 ,
       -55.919586, -54.760098, -54.88868 , -54.7405  , -54.12242 ,
       -54.520367, -54.005093, -53.664368, -53.864197, -54.86297 ,
       -54.40808 , -54.282944, -54.722507, -55.883984, -55.695946,
       -55.857346, -56.58976 , -58.016827, -59.17675 , -60.467514,
       -61.3338  , -61.48632 , -61.48632 , -61.48632 , -61.48632 ,
       -61.48632 , -61.48632 , -61.48632 , -61.48632 , -61.48632 ,
       -61.4063  , -59.998955, -41.15652 , -33.721714, -33.003082,
       -28.2406  , -24.803522, -25.940857, -27.145533, -27.862942,
       -28.898512, -29.746517, -30.327833, -30.533176, -30.471928,
       -29.964096, -28.690737, -27.944588, -26.04714 , -21.902466,
       -20.894138, -20.668617, -21.73722 , -23.520123, -24.755423,
       -25.363684, -26.322096, -27.510904, -28.95077 , -30.434858,
       -32.192398, -33.47311 , -32.413536, -32.973305, -36.774

# ConCatenating all the columns together.

In [17]:
# TURN ARRAY INTO LIST AND JOIN WITH AUDIO_DF TO GET CORRESPONDING EMOTION LABELS
df_combined = pd.concat([audio_df,pd.DataFrame(df['mel_spectrogram'].values.tolist())],axis=1)
df_combined = df_combined.fillna(0)

In [18]:
# DROP PATH COLUMN FOR MODELING
df_combined.drop(columns='Path',inplace=True)
df_combined.head()

,Gender,Emotion,Actor_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258
0,male,angry,1,-64.176788,-64.393738,-64.444954,-64.268044,-64.303825,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.447296,-64.412201,-64.400055,-64.463631,-64.467010,-64.447456,-64.398003,-64.213005,-64.223236,-64.352898,-64.377380,-63.900597,-63.455616,-63.705559,-63.493465,-63.146347,-63.393356,-64.024376,-64.014107,...,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.446861,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.468147,-64.422607,-64.364937,-64.468147,-64.465858,-64.354759,-64.385941
1,male,calm,1,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.438843,-78.454201,-78.454201,-78.454201,-78.410423,-78.450684,-78.454201,-78.423790,-78.208931,-78.261787,-78.393005,-78.181946,-76.545563,-77.805222,-78.356667,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.358124,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.454201,-78.303322,-77.402367,...,-77.662689,-77.959778,-77.530724,-77.818649,-78.392990,-78.245819,-78.109596,-77.947098,-77.856438,-78.035370,-77.977028,-78.129005,-78.303757,-78.454201,-78.370880,-78.163841,-78.345741,-78.445099,-78.454201,-78.454201,-78.364929,-78.218132,-78.112137,-78.430374,-78.454201,-78.053932,-77.932953,-78.429512,-78.454201,-78.211533,-78.155396,-78.425537,-78.454201,-78.454201,-78.454201,-78.441124,-78.454201,0.000000,0.000000,0.000000
2,male,sad,1,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.229431,-68.502647,-68.048340,-66.714714,-66.569191,-66.507698,-66.301208,-66.074875,-67.552040,...,-69.638710,-69.565544,-69.173370,-69.702393,-69.707024,-69.668999,-69.649261,-69.707024,-69.605957,-69.446609,-69.643822,-69.707024,-69.707024,-69.707024,-69.707024,-69.644974,-69.521240,-69.545723,-69.706207,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024,-69.707024
3,male,fear,1,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.255249,-73.086319,-73.234695,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.280296,-73.276604,-72.743233,-71.090782,-70.324883,-69.201920,-68.223709,-67.453972,-66.758484,-65.694832,-65.750854,-66.724770,-66.233826,-64.321724,-60.120052,...,-66.891136,-66.120682,-65.074013,-64.953125,-65.560944,-66.357735,-67.087402,-65.717964,-64.857964,-65.404991,-65.109146,-65.750305,-66.323601,-67.260437,-68.170120,-69.806412,-69.922493,-69.255791,-69.982750,-69.889183,-70.506737,-70.457909,-70.751228,-70.714455,-71.486320,-72.599976,-72.950294,-72.629456,-72.750542,-72.966499,-72.866135,-73.029083,-73.021973,-73.157417,-73.278503,-73.280296,-73.272202,-73.042091,-72.969475,-73.264915
4,male,sad,1,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.051636,-76.397690,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-76.469704,-

In [19]:
df_combined.tail(5)

,Gender,Emotion,Actor_ID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258
1435,female,surprise,24,-45.199844,-43.823120,-44.344078,-45.735817,-46.095490,-45.702011,-45.733891,-46.317333,-47.424995,-48.189751,-48.735367,-49.609917,-51.114033,-51.741745,-52.354424,-54.285316,-55.315136,-56.219372,-57.399914,-58.403381,-58.453133,-59.335815,-61.146862,-61.302704,-61.302704,-61.302704,-61.302704,-61.302704,-61.302704,-61.302704,-61.302704,-61.302704,-61.302704,-61.243439,-61.220222,-61.302704,-61.302704,...,-60.969559,-60.875137,-60.722389,-60.118011,-60.095760,-60.399910,-60.534489,-60.654854,-60.619862,-60.413250,-60.633583,-60.580307,-60.691219,-60.813469,-60.829540,-60.331841,-60.297447,-60.833664,-60.817112,-60.761837,-61.069309,-61.046097,-60.780121,-60.577831,-61.091087,-60.945629,-60.712101,-60.982864,-61.139175,-61.273045,-61.288826,-61.167828,-61.031166,-60.674316,-60.724525,-60.864182,-60.995823,-61.174706,-61.302704,-61.302704
1436,female,surprise,24,-68.403206,-68.403206,-68.403206,-68.403206,-68.403206,-68.403206,-68.403206,-68.403206,-68.399330,-67.857368,-67.090279,-66.301811,-66.843895,-64.562645,-62.644104,-61.084389,-51.850674,-46.687138,-46.233536,-47.738266,-50.351227,-51.099136,-51.558739,-51.873337,-51.094936,-52.011761,-53.007988,-53.324146,-54.204929,-56.115910,-58.938377,-60.911224,-62.974407,-65.346306,-66.777313,-66.880432,-67.607178,...,-68.365784,-68.357353,-68.364586,-68.403206,-68.379295,-68.334869,-68.215965,-68.269455,-68.292999,-68.347565,-68.403206,-68.403206,-68.403206,-68.403206,-68.323814,-68.282799,-68.355751,-68.350388,-68.331108,-68.380005,-68.360817,-68.217102,-68.400757,-68.403206,-68.403206,-68.403206,-68.371307,-68.205475,-68.298798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1437,female,disgust,24,-68.237564,-67.372169,-66.390640,-66.679070,-67.211220,-66.475182,-65.987640,-64.506271,-63.091972,-62.799042,-62.180588,-62.196743,-62.990387,-63.585327,-63.532772,-64.269554,-55.129112,-53.418171,-60.280621,-62.336689,-64.341957,-66.673210,-68.575859,-69.063072,-69.530563,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,-69.540901,...,-49.840218,-50.434002,-49.469864,-46.537342,-44.396843,-43.032753,-41.553986,-40.733849,-40.363548,-40.072964,-40.077297,-39.949459,-40.815861,-41.772476,-42.216290,-43.267544,-43.708328,-43.457760,-44.937130,-45.133762,-48.862083,-46.105080,-48.322117,-46.981079,-46.033066,-47.234325,-47.646309,-47.781994,-49.038277,-50.453804,-51.462769,-51.906311,-53.411156,-54.769276,-55.461460,-56.681473,-58.014870,-58.699448,-58.785629,-59.667213
1438,female,disgust,24,-59.609074,-59.128216,-59.562916,-61.302921,-61.619808,-61.774040,-62.131512,-62.654728,-62.494194,-62.513931,-62.470318,-62.602581,-62.404499,-62.136993,-61.945717,-61.070309,-60.944786,-60.753075,-60.514732,-60.632572,-61.278114,-61.728016,-61.656292,-62.304691,-62.698635,-62.698635,-62.698635,-62.698635,-62.698635,-62.698635,-62.698635,-62.698635,-62.698635,-62.698635,-62.668308,-62.698635,-62.190578,...,-39.426277,-39.130020,-37.793678,-36.584705,-36.504169,-36.978874,-36.565567,-35.823742,-35.471481,-34.081585,-33.161747,-33.365112,-32.182667,-31.232910,-30.743446,-31.072195,-31.652185,-31.807903,-31.547956,-32.071133,-33.142200,-34.153030,-34.562847,-34.311996,-35.252117,-36.341122,-37.647221,-37.705730,-38.326855,-38.874119,-39.059551,-39.546028,-39.909897,-41.021183,-42.025139,-41.798611,-41.806911,-42.457855,-43.402187,-45.924404
1439,female,disgust,24,-59.715309,-59.487293,-58.765045,-58.078152,-57.865799,-57.900166,-56.573082,-57.192883,-57.220207,-56.616772,-55.919586,-54.760098,-54.888680,-54.740501,-54.122421,-54.520367,-54.005093,-53.664368,-53.864197,-5

# Creating train and test data

In [20]:
# TRAIN TEST SPLIT DATA
from sklearn.model_selection import train_test_split
train,test = train_test_split(df_combined, test_size=0.2, random_state=0,
                               stratify=df_combined[['Emotion','Gender','Actor_ID']])

In [21]:
#TRAINING SET
X_train = train.iloc[:, 3:] #Considering only the features here (they start from the 3rd column.)
y_train = train.iloc[:,:2].drop(columns=['Gender'])#Since we need only want to identify the emotion,we have used this.
print(X_train.shape)
print(X_train)

#TESTING SET
X_test = test.iloc[:,3:]
y_test = test.iloc[:,:2].drop(columns=['Gender'])
print(X_test.shape)
print(X_test)

(1152, 259)
              0          1          2  ...        256        257        258
703  -70.913582 -70.885597 -71.084183  ... -69.723839 -70.186195 -70.453629
459  -67.897774 -67.764366 -67.827827  ... -67.956894 -67.528030 -67.378983
1300 -68.620384 -68.490128 -68.540581  ... -68.801735 -68.801735 -68.801735
1436 -68.403206 -68.403206 -68.403206  ...  0.000000   0.000000   0.000000 
893  -65.981171 -65.690796 -65.279762  ...  0.000000   0.000000   0.000000 
...         ...        ...        ...  ...       ...        ...        ... 
1229 -48.510590 -48.510590 -48.510590  ... -48.510590 -48.510590 -48.505608
1132 -67.904999 -67.943665 -68.031281  ... -67.876793 -67.729774 -67.574707
2    -69.707024 -69.707024 -69.707024  ... -69.707024 -69.707024 -69.707024
603  -72.728462 -70.554390 -70.852989  ...  0.000000   0.000000   0.000000 
32   -67.132706 -67.193504 -67.123268  ... -67.962799 -68.024063 -68.404770

[1152 rows x 259 columns]
(288, 259)
              0          1          2 

In [22]:
# NORMALIZE DATA
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# ONE-HOT Encoding

In [24]:

# ONE HOT ENCODE THE TARGET
# CNN REQUIRES INPUT AND OUTPUT ARE NUMBERS
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_test = to_categorical(lb.fit_transform(y_test))

print(y_test[0:10])
#Since,there are 8 different emotions; we get 8 columns.
#The places where 1 is present represents that particular emotion.

[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]
